In [4]:
from pymongo import MongoClient
import pandas as pd

In [5]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection("mongoProjectDB", "companies")

In [4]:
rest = list(coll.find({"offices.country_code":"USA"}))


In [5]:
def getLocation(company):
    for i in range(len(company["offices"])):
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
    return loc

In [6]:
for company in rest:
    value = {"$set": {'location':getLocation(company)}}
    coll.update_one(company,value)

In [7]:
for company in rest:
    value = {"$unset": {'offices.latitude':''}}
    value = {"$unset": {'offices.longitude':''}}
    coll.update_one(company,value)

In [8]:
pipeline = [
    {"$unwind":"$offices"},
    {"$match":{
        "$and":[
            {"offices.country_code":"USA"},
            {'$or':[
                {'offices.latitude':{'$ne':None}},
                {'offices.longitude':{'$ne':None}}
            ]},
            {"funding_rounds.raised_amount":{
                "$gte":1000000
            }},
            {"deadpooled_year":None}
        ]
    }
    }
]

In [9]:
query = list(coll.aggregate(pipeline))

In [10]:
filtered_df = pd.DataFrame(query)

In [11]:
filtered_df.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'alias_list', 'email_address',
       'phone_number', 'description', 'created_at', 'updated_at', 'overview',
       'image', 'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'acquisition',
       'acquisitions', 'offices', 'milestones', 'ipo', 'video_embeds',
       'screenshots', 'external_links', 'partners', 'location'],
      dtype='object')

In [10]:
for i in filtered_df["location"]:
    if i == {"type": "Point", "coordinates": [None, None]}:
        print(i)
        break

NameError: name 'filtered_df' is not defined

In [13]:
filtered_df = filtered_df.loc[filtered_df["location"] != {'type': 'Point', 'coordinates': [None, None]}]

In [14]:
filtered_df = filtered_df.drop(["_id"], axis=1)

In [15]:
filtered_df["location"]

0       {'type': 'Point', 'coordinates': [-118.393064,...
1       {'type': 'Point', 'coordinates': [-122.394523,...
2       {'type': 'Point', 'coordinates': [-122.169472,...
3       {'type': 'Point', 'coordinates': [-117.081406,...
4       {'type': 'Point', 'coordinates': [-122.154369,...
                              ...                        
2405    {'type': 'Point', 'coordinates': [-117.607717,...
2406    {'type': 'Point', 'coordinates': [-122.181753,...
2407    {'type': 'Point', 'coordinates': [-121.92917, ...
2408    {'type': 'Point', 'coordinates': [-121.998513,...
2409    {'type': 'Point', 'coordinates': [-122.151599,...
Name: location, Length: 2143, dtype: object

In [16]:
cities_df = []
for e in filtered_df["offices"]:
    cities_df.append(e["city"])


In [17]:
cities_df = set(cities_df)

In [18]:
cities_df.remove("")
cities_df.remove(None)

In [19]:
cities_df = list(cities_df)

In [20]:
len(cities_df)

415

In [21]:
#cities_df.remove("Tartu")

In [22]:
%store cities_df
len(cities_df)

Stored 'cities_df' (list)


415

In [23]:
filtered_df["location"][0]

{'type': 'Point', 'coordinates': [-118.393064, 34.090368]}

In [24]:
filtered_df.to_json("./output/companies_clean.json", orient="records")

In [35]:
db, companies_coll = connectCollection("mongoProjectDB", "companies_clean")
q = list(companies_coll.find())
companies_df = pd.DataFrame(q)

In [26]:
companies_df = pd.concat([companies_df.drop(['location'], axis=1), companies_df['location'].apply(pd.Series)], axis=1)

In [82]:
db, starbucks_coll = connectCollection("mongoProjectDB", "starbucks")
db, schools_coll = connectCollection("mongoProjectDB", "schools")
db, vegan_coll = connectCollection("mongoProjectDB", "vegan")
db, airports_coll = connectCollection("mongoProjectDB", "airports")
db, companies_coll = connectCollection("mongoProjectDB", "companies_clean")
starbucks_q = list(starbucks_coll.find())
schools_q = list(schools_coll.find())
vegan_q = list(vegan_coll.find())
airports_q = list(airports_coll.find())
companies_q = list(companies_coll.find())

In [83]:
def geoquery(geoindex,distance,col):
    points= col.find(
        {"location":
         {"$near":
          {"$geometry":
           geoindex,
           "$maxDistance":distance
            }
        }
    })
    return list(points)

In [84]:
geo_starbucks = starbucks_q[0]["Geometry"]

In [85]:
geoquery(geo_starbucks, 1000, companies_coll)[0]["offices"]["latitude"]

43.068006

In [115]:
latitude = []
longitude = []
for i in range(len(companies_q)):
    latitude.append(geoquery(geo_starbucks, 1000, companies_coll)[0]["offices"]["latitude"])
    longitude.append(geoquery(geo_starbucks, 1000, companies_coll)[0]["offices"]["longitude"])



[43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.068006,
 43.

In [121]:
companies_q[0]["location"][]

{'type': 'Point', 'coordinates': [-122.169472, 37.48413]}

In [122]:
geoquery(geo_starbucks, 1000, companies_coll)[0]

{'_id': ObjectId('5dd6f9fd2db003cc387280fa'),
 'name': 'StudyBlue',
 'permalink': 'studyblue',
 'crunchbase_url': 'http://www.crunchbase.com/company/studyblue',
 'homepage_url': 'http://www.studyblue.com',
 'blog_url': 'http://www.studyblue.com/about/blog/',
 'blog_feed_url': 'http://feeds.feedburner.com/studyblue',
 'twitter_username': 'studyblue',
 'category_code': 'education',
 'number_of_employees': 18.0,
 'founded_year': 2009.0,
 'founded_month': None,
 'founded_day': None,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'education, study-tool, flashcards, notes, sharing, academic-network',
 'alias_list': '',
 'email_address': 'info@studyblue.com',
 'phone_number': '',
 'description': 'StudyBlue is your online home to store l',
 'created_at': 'Tue Nov 11 05:21:17 UTC 2008',
 'updated_at': 'Wed Dec 04 14:01:23 UTC 2013',
 'overview': '<p>StudyBlue is made for students. Its mobile and social study platform empowers 

In [113]:
geoquery(geo_starbucks, 1000, companies_coll)[0]["offices"]["longitude"]

-89.39256